# 패키지

In [1]:
# 필요한 패키지
import os                           # 운영체제
import time                         # 시간
import tensorflow as tf             # 텐서플로
import tensorflow_addons as tfa     # 텐서플로 에드온
import pandas as pd                 # 판다스
import matplotlib.pyplot as plt     # 그래프 도구
import MSRL                         # Musical Symbol Recognition Library

c:\Users\chorokmoo_win\.conda\envs\new\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [53]:
y_true = tf.constant([[10, 10, 50, 50, 30, 30, 20, 20, 60, 60, 40, 40, 0, 0, 0, 0, 0, 0]], dtype=tf.int16)
y_pred = tf.constant([[47, 15, 15, 48, 31, 31, 20, 20, 60, 60, 40, 40, 0, 7, 8, 0, 0, 0]], dtype=tf.float32)

In [54]:
y_true = tf.cast(y_true, dtype=y_pred.dtype)

In [55]:
y_true = tf.reshape(y_true, shape=(-1, 3, 6))
y_pred = tf.reshape(y_pred, shape=(-1, 3, 6))

In [50]:
# 0. 좌표 정렬
def sort_coordinate(box):
    # 각각의 좌표로 나누기
    x1 = box[:, :, 0:1]
    y1 = box[:, :, 1:2]
    x2 = box[:, :, 2:3]
    y2 = box[:, :, 3:4]
    # 좌표 크기 비교
    new_x1 = tf.minimum(x1, x2)
    new_y1 = tf.minimum(y1, y2)
    new_x2 = tf.maximum(x1, x2)
    new_y2 = tf.maximum(y1, y2)
    # 텐서 하나로 만들기
    return tf.concat([new_x1, new_y1, new_x2, new_y2], axis=-1)

# 1-1. 박스 넓이 계산
def box_area(box):
    # 각각의 좌표로 나누기
    x1 = box[:, :, 0:1]
    y1 = box[:, :, 1:2]
    x2 = box[:, :, 2:3]
    y2 = box[:, :, 3:4]
    # 넓이 계산
    return tf.abs((x2-x1) * (y2-y1))

# 1-2. IoU 계산
def compute_IoU(box1, box2):
    # 예측 박스 좌표 정렬
    box2 = sort_coordinate(box2)

    # 교집합 영역의 박스 좌표 구함
    box3 = tf.concat([
        tf.maximum(box1[:, :, 0:1], box2[:, :, 0:1]),
        tf.maximum(box1[:, :, 1:2], box2[:, :, 1:2]),
        tf.minimum(box1[:, :, 2:3], box2[:, :, 2:3]),
        tf.minimum(box1[:, :, 3:4], box2[:, :, 3:4])
    ], axis=-1)
    
    # 교집합 영역의 넓이 계산
    intersection_area = box_area(box3)

    # 합집합 영역 계산
    union_area = box_area(box1) + box_area(box2) - intersection_area

    # 0~1 사이값 리턴
    return 1.0 - (intersection_area / union_area)

# 2-1. 중심점 계산
def get_center(box):
    # 각각의 좌표로 나누기
    x1 = box[:, :, 0:1]
    y1 = box[:, :, 1:2]
    x2 = box[:, :, 2:3]
    y2 = box[:, :, 3:4]
    # 중심점 계산
    cx = (x1 + x2) / 2.0
    cy = (y1 + y2) / 2.0
    return cx, cy

# 2-2. 박스 대각선 길이 계산
def box_diagonal(box):
    # 각각의 좌표로 나누기
    x1 = box[:, :, 0:1]
    y1 = box[:, :, 1:2]
    x2 = box[:, :, 2:3]
    y2 = box[:, :, 3:4]
    # 대각선 길이 계산
    return tf.sqrt((x2 - x1) ** 2 + (y2 -y1) ** 2)

# 2-3. 두 박스의 중심점 거리 계산 -> 0 이상의 값 리턴
def center_distance(box1, box2):
    # 예측 박스 좌표 정렬
    box2 = sort_coordinate(box2)

    # 두 박스의 중심점 사이 거리
    b1_cx, b1_cy = get_center(box1)
    b2_cx, b2_cy = get_center(box2)
    box_distance = tf.sqrt((b2_cx - b1_cx) ** 2 + (b2_cy - b1_cy) ** 2)

    # 두 박스를 모두 포함하는 합집합 영역의 대각선 길이
    box4 = tf.concat([
        tf.minimum(box1[:, :, 0:1], box2[:, :, 0:1]),
        tf.minimum(box1[:, :, 1:2], box2[:, :, 1:2]),
        tf.maximum(box1[:, :, 2:3], box2[:, :, 2:3]),
        tf.maximum(box1[:, :, 3:4], box2[:, :, 3:4])
    ], axis=-1) 
    union_diagonal = box_diagonal(box4)
    
    # 0~1 사이값 리턴
    return box_distance / union_diagonal

# 3-1. 각 박스의 너비와 높이 비율 계산 
def box_ratio(box):
    # 각각의 좌표로 나누기
    x1 = box[:, :, 0:1]
    y1 = box[:, :, 1:2]
    x2 = box[:, :, 2:3]
    y2 = box[:, :, 3:4]
    # 너비와 높이 계산
    width = tf.abs(x2 - x1)
    height = tf.abs(y2 - y1)
    return height / (width + 1e-6) # 0 나누기 방지

# 3-2. 종횡 비율 차이 계산
def ratio_differences(box1, box2):
    # 예측 박스 좌표 정렬
    box2 = sort_coordinate(box2)

    # 각 박스 비율 계산
    b1_ratio = box_ratio(box1)
    b2_ratio = box_ratio(box2)

    # 0~1 사이값 리턴
    return tf.abs(b1_ratio - b2_ratio) / tf.maximum(b1_ratio, b2_ratio)

# 4. 좌표 뒤바뀜 패널티 계산
def coordinate_penalty(box2):
    # 각각의 좌표로 나누기
    x1 = box2[:, :, 0:1]
    y1 = box2[:, :, 1:2]
    x2 = box2[:, :, 2:3]
    y2 = box2[:, :, 3:4]

    # 예측 박스 좌표 바뀜 패널티 계산
    penalty_x = tf.maximum(0.0, x1 - x2)
    penalty_y = tf.maximum(0.0, y1 - y2)

    # 0~1 사이값 리턴
    return penalty_x + penalty_y

# 5-1. IoU 손실 구하기
def iou_loss(y_true, y_pred):
    # 바운딩 박스 좌표와 상대적 중심 좌표 분리
    box_true = y_true[:, :, 0:4]
    box_pred = y_pred[:, :, 0:4]
    point_true = y_true[:, :, 4:6]
    point_pred = y_pred[:, :, 4:6]

    # CIoU = IoU + 중심점 거리 + 종횡비 + 좌표 뒤바낌 패널티
    iou = compute_IoU(box_true, box_pred)          # 0~1 : IoU 계산
    distance = center_distance(box_true, box_pred) # 0~1 : 중심점 거리 계산
    ratio = ratio_differences(box_true, box_pred)  # 0~1 : 박스 비율 차이 계산
    penalty = coordinate_penalty(box_pred)         # 0~n :좌표 뒤바뀜 패널티

    # 중심 좌표 손실
    center_loss = box_diagonal(tf.concat([point_true, point_pred], axis=-1)) / box_diagonal(box_true)
    
    # 손실 계산
    # : (batch_size, class_count, 1)
    return iou + distance + ratio + penalty + center_loss

# 5-2. 0 수렴 손실 구하기
def to_zero_loss(y_pred):
    # 모든 좌표 절대값 취하기
    y_pred = tf.abs(y_pred)

    # 로그 손실 구하기
    loss = tf.math.log(y_pred + 1.0)

    # 손실 합 반환
    return tf.reduce_sum(loss, axis=-1, keepdims=True)

In [163]:
def call(y_true, y_pred):
    # 자료형 통일
    y_true = tf.cast(y_true, dtype=y_pred.dtype)

    # 배치 크기 알아내기
    batch_size = tf.shape(y_true)[0]

    # shape 변경
    y_true = tf.reshape(y_true, shape=(batch_size, -1, 6))
    y_pred = tf.reshape(y_pred, shape=(batch_size, -1, 6))

    # 바운딩 박스 좌표와 상대적 중심 좌표 분리
    box_true = y_true[:, :, 0:4]
    
    # 손실 계산
    loss = tf.where(box_area(box_true)==0, to_zero_loss(y_pred), iou_loss(y_true, y_pred))

    # 소수 레이블에 더 많은 가중치를 부여
    # : (batch_size, class_count, 1)
    weightedLoss = loss

    # 각 클래스마다 계산된 손실 합산
    # : (batch_size,)
    sumLoss = tf.reduce_sum(weightedLoss, axis=[-1, -2])

    # 배치 크기만큼 손실 평균
    # : ()
    meanLoss = tf.reduce_mean(sumLoss, axis=-1)

    # 손실 반환
    return meanLoss

# 모델 1

In [ ]:
# 데이터셋 준비
ds_1_train, ds_1_validation = msDataset.ds_1()

-- TFDS label shape ------------------
ds 0  : (22617, 133)

-- TFDS label class count ------------------
ds 0  : [1175, 1193, 1224, 1228, 1224, 1074, 537, 524, 544, 553, 1059, 1092, 1124, 1060, 1, 1, 1, 1, 1, 1, 1, 1, 1, 738, 745, 722, 748, 744, 746, 720, 750, 737, 702, 725, 724, 774, 750, 725, 722, 765, 719, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 23, 1, 1, 11, 11, 11, 1655, 1612, 1622, 1585, 1619, 1582, 1641, 1589, 1516, 1622, 1639, 1626, 1587, 1607, 981, 1054, 1054, 987, 1047, 1035, 1036, 986, 1080, 1070, 1120, 1067, 1084, 1088, 1109, 5, 5, 5, 5, 1, 1, 1, 5, 5, 1968, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 47, 2017, 2036, 2073, 2055, 2058, 2117, 2082, 1978, 2025, 2022, 2050, 34, 23, 1, 1]

-- MODEL input node ------------------
[[22617, None, None, 1]]

-- MODEL output node ------------------
[[133]]


In [ ]:
# 그래프 초기화
tf.keras.backend.clear_session()

with tf.device('/device:GPU:0'):
    # 모델 존재 확인
    if os.path.exists(model_dir_1): model_1 = tf.keras.models.load_model(model_dir_1)   # 모델 불러오기
    else                          : model_1 = msModel.model_1()                         # 모델 생성

    # 모델 학습 설정
    #model_1.compile()

    # 모델 훈련 및 저장
    #history_1 = model_1.fit(
    #    ds_1_train, 
    #    epochs=20, 
    #    initial_epoch=0, 
    #    validation_data=ds_1_validation, 
    #    callbacks=[cb_checkpoint_1, cb_early_stop]
    #)

# 모델 2

In [ ]:
# 모델 주소
model_dir_2 = os.path.join('.', 'models', 'model_2_CNN_MHA.h5')
model_dir_epoch_2 = os.path.join('.', 'models', 'model_2_CNN_MHA-{epoch}.h5')
tensorboard_dir = os.path.join('.', 'model_TB', 'model_2_CNN_MHA', time.strftime("%Y_%m_%d-%H_%M_%S"))

# 모델 콜백 함수
cb_checkpoint_2 = tf.keras.callbacks.ModelCheckpoint(model_dir_epoch_2, monitor='loss', save_best_only=True)
cb_early_stop_2 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, restore_best_weights=False)
cb_tensorboard = tf.keras.callbacks.TensorBoard(tensorboard_dir)

# 데이터셋 준비
ds_2_train, ds_2_validation = msDataset.ds_2()
#ds_2_train, ds_2_validation, ds_2_test = msDataset.ds_2()  # 추후 이런 식으로 변경할 예정

-- TFDS label shape ------------------
ds 0  : (22617, 12)

-- TFDS label class count ------------------
ds 0  : [22502, 6044, 7567, 13256, 8180, 7618, 2011, 9, 14, 58, 15, 2]

-- MODEL input node ------------------
[[22617, None, None, 1]]

-- MODEL output node ------------------
[[12]]


In [ ]:
# 그래프 초기화
tf.keras.backend.clear_session()

with tf.device('/device:GPU:0'):
    # 모델 존재 확인
    if os.path.exists(model_dir_2): model_2 = tf.keras.models.load_model(model_dir_2) # 모델 불러오기
    else:                           model_2 = msModel.model_2_CNN_MHA() # 모델 생성

    # 모델 컴파일 설정
    model_2.compile(
        optimizer='adam',
        loss=MSRL.losses.WeightedBinaryCrossentropy(22617, [22502, 6044, 7567, 13256, 8180,  7618, 2011, 9, 14, 58, 15, 2]),
        metrics=[
            MSRL.metrics.Accuracy(),                                # 정확도
            MSRL.metrics.HammingScore(),                            # 해밍 점수
            #tf.keras.metrics.Precision(),                           # 정밀도
            #tf.keras.metrics.Recall(),                              # 재현율
            #tfa.metrics.F1Score(num_classes=12, average='macro')    # f1-점수
        ]
    )

    # 모델 훈련
    history_2 = model_2.fit(
        ds_2_train,
        epochs=20,
        initial_epoch=0,
        validation_data=ds_2_validation,
        callbacks=[cb_checkpoint_2, cb_early_stop_2, cb_tensorboard]
    )

    # 현재까지 학습된 모델 저장
    model_2.save(model_dir_2)

# 텐서보드 확인
# tensorboard.exe --logdir=.\model_TB\model_2_CNN_MHA --port=6006

Epoch 1/20
697/697 [==============================] - 216s 266ms/step - loss: 3.2210 - Accuracy: 0.4159 - HammingScore: 0.9288 - val_loss: 3.5726 - val_Accuracy: 0.3250 - val_HammingScore: 0.9203
Epoch 2/20
697/697 [==============================] - 178s 256ms/step - loss: 1.8233 - Accuracy: 0.6284 - HammingScore: 0.9621 - val_loss: 43.6198 - val_Accuracy: 0.0500 - val_HammingScore: 0.7339
Epoch 3/20
697/697 [==============================] - 176s 252ms/step - loss: 1.3650 - Accuracy: 0.7219 - HammingScore: 0.9726 - val_loss: 2.1008 - val_Accuracy: 0.6562 - val_HammingScore: 0.9617
Epoch 4/20
697/697 [==============================] - 176s 252ms/step - loss: 1.6768 - Accuracy: 0.6830 - HammingScore: 0.9653 - val_loss: 1.3880 - val_Accuracy: 0.6719 - val_HammingScore: 0.9701
Epoch 5/20
697/697 [==============================] - 175s 251ms/step - loss: 1.2573 - Accuracy: 0.7244 - HammingScore: 0.9741 - val_loss: 3.8749 - val_Accuracy: 0.4125 - val_HammingScore: 0.9297
Epoch 6/20
697/697 

# 모델 3

In [ ]:
# 데이터셋 준비
ds_3_train, ds_3_validation = msDataset.ds_3()

-- TFDS label shape ------------------
ds 0  : (22617, 12)
ds 1  : (22617, 11)
ds 2  : (22617, 14)
ds 3  : (22617, 5)
ds 4  : (22617, 9)
ds 5  : (22617, 18)
ds 6  : (22617, 8)
ds 7  : (22617, 7)
ds 8  : (22617, 12)
ds 9  : (22617, 9)
ds 10 : (22617, 14)
ds 11 : (22617, 6)
ds 12 : (22617, 15)
ds 13 : (22617, 2)

-- TFDS label class count ------------------
ds 0  : [22502, 6044, 7567, 13256, 8180, 7618, 2011, 9, 14, 58, 15, 2]
ds 1  : [2017, 2036, 2073, 2055, 2058, 2117, 2082, 1978, 2025, 2022, 2050]
ds 2  : [1655, 1612, 1622, 1585, 1619, 1582, 1641, 1589, 1516, 1622, 1639, 1626, 1587, 1607]
ds 3  : [1175, 1193, 1224, 1228, 1224]
ds 4  : [1074, 537, 524, 544, 553, 1059, 1092, 1124, 1060]
ds 5  : [738, 745, 722, 748, 744, 746, 720, 750, 737, 702, 725, 724, 774, 750, 725, 722, 765, 719]
ds 6  : [981, 1054, 1054, 987, 1047, 1035, 1036, 986]
ds 7  : [1080, 1070, 1120, 1067, 1084, 1088, 1109]
ds 8  : [5, 5, 5, 5, 1, 1, 1, 5, 5, 1968, 5, 5]
ds 9  : [1, 1, 1, 1, 1, 1, 1, 1, 1]
ds 10 : [1, 1, 1,

In [ ]:
# 그래프 초기화
tf.keras.backend.clear_session()

with tf.device('/device:GPU:0'):
    # 모델 존재 확인
    if os.path.exists(model_dir_3): model_3 = tf.keras.models.load_model(model_dir_3)   # 모델 불러오기
    else                          : model_3 = msModel.model_3()                         # 모델 생성

    # 모델 학습 설정
    #model_3.compile()

    # 모델 훈련 및 저장
    #history_3 = model_3.fit(
    #    ds_3_train, 
    #    epochs=20, 
    #    initial_epoch=0, 
    #    validation_data=ds_3_validation, 
    #    callbacks=[cb_checkpoint_3, cb_early_stop]
    #)

Epoch 1/10
697/697 [==============================] - 169s 177ms/step - loss: 0.4941 - all_loss: 0.0237 - pitch_loss: 0.1660 - note_loss: 0.0711 - accidental_loss: 0.0536 - articulation_loss: 0.0334 - dynamic_loss: 0.0326 - octave_loss: 0.0453 - ornament_loss: 0.0482 - repetition_loss: 1.7628e-04 - clef_loss: 1.1788e-04 - key_loss: 9.8481e-05 - measure_loss: 0.0193 - rest_loss: 1.2383e-04 - time_loss: 3.9371e-04 - all_accuracy: 3.5879e-04 - pitch_accuracy: 0.0893 - note_accuracy: 0.0783 - accidental_accuracy: 0.7804 - articulation_accuracy: 0.7085 - dynamic_accuracy: 0.4432 - octave_accuracy: 0.6722 - ornament_accuracy: 0.7067 - repetition_accuracy: 0.9056 - clef_accuracy: 0.9983 - key_accuracy: 0.9922 - measure_accuracy: 0.9956 - rest_accuracy: 0.9936 - time_accuracy: 0.9985 - val_loss: 0.4814 - val_all_loss: 0.0231 - val_pitch_loss: 0.1634 - val_note_loss: 0.0710 - val_accidental_loss: 0.0506 - val_articulation_loss: 0.0351 - val_dynamic_loss: 0.0319 - val_octave_loss: 0.0406 - val_o